# NLP Test
Testing NLP pipeline and preprocessing.

- **Input** 
    - Crawler's output that stored in SQL database.
    
- **Ouput**
    - Preprocessed crawled data and prepare json for meilisearch indexing.
    
- **Processes**
    - Load Data from SQL
    - Extract Content from HTML
        - extract code block and content text from HTML
    - Preprocess Content Text
        - word segmentation and stop word removal
    - Keyword Extraction from Processed Contents
    - Format Json for Meilisearch Indexing

## Parameters

### Database File
- "testing/testing-ironman_100.db"

### Database Access Query**
```sql
SELECT 
    a.href,
    a.title, 
    a.content AS content_html, 
    a.tags AS raw_tags_string, 
    a.genre, 
    a.publish_at AS published_at,
    a.author_href, 
    u.name AS author_name,
    a.series_href, 
    s.name AS series_name, 
    a.series_no AS series_num
FROM articles a
LEFT JOIN users u 
ON a.author_href = u.href
LEFT JOIN series s
ON a.series_href = s.href 
LIMIT 10;
```

### Resource Files for Preprocessing 
- "resources/jieba_dict.txt.big"
- "resources/jieba_stop_words.txt"
- "resources/puntuation_mark.list"
- "resources/traditional_chinese_stop_list.txt"


### Deprecated
```python
def __sql_row_to_info(row):
    return {
        'href':         row[0],
        'title':        row[1],
        'content_html': row[2],
        'tags':         row[3],
        'genre':        row[4],
        'published_at': row[5],
        'series_href':  row[7],
        'series_no':    row[8],
        'series_name':  row[9],
    }
```

In [1]:
import os
import json
import time, datetime
from typing import Callable

import sqlite3
from bs4 import BeautifulSoup
from keybert import KeyBERT

## Resources 
Load resources files such as stop word list and jieba dictionary for jieba word segmenter.

In [2]:
import jieba

nlp_root = os.path.dirname(os.getcwd())

jieba.set_dictionary(os.path.join(nlp_root, "resources/jieba_dict.txt.big"))
jieba.initialize()


merged_stopword_set = set()

with open(os.path.join(nlp_root, "resources/jieba_stop_words.txt"), 'r') as infile: 
    for line in infile:
        line = line.strip()
        if line: 
            merged_stopword_set.add(line)
        
with open(os.path.join(nlp_root, "resources/puntuation_mark.list"), 'r') as infile: 
    for line in infile:
        line = line.strip()
        if line: 
            merged_stopword_set.add(line)

with open(os.path.join(nlp_root, "resources/traditional_chinese_stop_list.txt"), 'r') as infile: 
    for line in infile:
        line = line.strip()
        if line: 
            merged_stopword_set.add(line)

Building prefix dict from /Users/lynch/Workspace/projects/over-engineering/nlp/resources/jieba_dict.txt.big ...
Loading model from cache /var/folders/j2/tldcg8p57f571z0hz8dt6s_00000gn/T/jieba.ucd8d8985486883511d8a6bcdc76e8683.cache
Loading model cost 0.532 seconds.
Prefix dict has been built successfully.


## Preprocessing 1: Load Data from SQL

In [3]:
def preprocessing_load_data_from_sql(db_path: str, sql_query: str, uid_name: str) -> dict: 
    
    # create a SQL connection to our SQLite database
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    
    # select articles left join series and users
    raw_data = cur.execute(sql_query)

    col_names = []
    for idx, col in enumerate(cur.description):
        col_names.append(col[0])
    
    info_dict = {}
    for row in raw_data:
        info = { k:v.strip() for k, v in zip(col_names, row) }
        info_dict[info[uid_name]] = info
    
    return info_dict    

In [4]:
db_path = os.path.join(nlp_root, "testing/testing-ironman_100.db")
sql_query = """
SELECT 
    a.href,
    a.title, 
    a.content AS content_html, 
    a.tags AS raw_tags_string, 
    a.genre, 
    a.publish_at AS published_at,
    a.author_href, 
    u.name AS author_name,
    a.series_href, 
    s.name AS series_name, 
    a.series_no AS series_num
FROM articles a
LEFT JOIN users u 
ON a.author_href = u.href
LEFT JOIN series s
ON a.series_href = s.href 
LIMIT 10;
"""
info_dict = preprocessing_load_data_from_sql(db_path, sql_query, uid_name="href")

# print debug
print_item_key = list(info_dict.keys())[4]
print(json.dumps(info_dict[print_item_key], indent=4, ensure_ascii=False))

{
    "href": "https://ithelp.ithome.com.tw/articles/10282236",
    "title": "[專案上線第01天] -  新來的主管說要寫 Vue Test Utils 單元測試",
    "content_html": "<div class=\"markdown__style\">\n                                                            <h3>前言</h3>\n<blockquote>\n<p>該系列是為了讓看過Vue官方文件或學過Vue但是卻不知道怎麼下手去重構現在有的網站而去規畫的系列文章，在這邊整理了許多我自己使用Vue重構很多網站的經驗分享給讀者們。</p>\n</blockquote>\n<p>什麼？單元測試？當你開始接觸開發專案有一段時間後，你會開始漸漸聽到這個專業術語，就讓我來大家了解一下什麼是單元測試</p>\n<p><iframe width=\"560\" height=\"315\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen=\"allowfullscreen\" src=\"https://www.youtube.com/embed/j2ggBXF54dA\"></iframe><br>\n影片搭配文章看學習效果會更好喔</p>\n<h2>什麼是單元測試？</h2>\n<p>簡單來說程式碼的最小單位進行測試，確保程式邏輯不會在團隊維護的過程中出錯，維護程式碼的品質。所謂的最小單位，我用個例子來舉例，假如你今天有一個主功能是由 A跟Ｂ兩個功能所組成的，而這兩個功能就是我們所說的最小單位，所以在撰寫測試的時候我們重點在針對A跟Ｂ來進行測試，主功能的測試中不會包含 A跟Ｂ的測試，這樣的測試就是我們所說的單元測試。</p>\n<h2>為什麼需要單元測試？</h2>\n<p>我先列出幾個優缺點，我們來比較一下</p>\n<h3>優點：</h3>\n<ol>\n<li>確保團隊跌代的時候不會影響原本的功能</li>\n<li>確保品質

## Preprocessing 2: Extract Content from HTML
- parse html and extract content text
- remove 'a' and 'img'
- extract codes from content

In [5]:
def preprocessing_extract_html(info: dict, html: str) -> dict:
    
    # parse html
    soup = BeautifulSoup(html, features="html.parser")
    
    # remove 'a' and 'img'
    for s_a in soup('a'):
        s_a.decompose()
    for s_img in soup('img'): 
        s_img.decompose()
    
    # extract codes from content
    code_list = []
    for s_pre in soup('pre'): 
        for s_code in s_pre('code'): 
            code_list.append(s_code.get_text())
            s_code.decompose()
            
    # result
    extracted_info = {
        'processed_content_html': str(soup),
        'content_text':           soup.get_text().strip(),
        'content_codes':          code_list,
    }
    
    return extracted_info

In [6]:
for k in info_dict:   
    extracted_info = preprocessing_extract_html(info_dict[k], info_dict[k]['content_html'])
    info_dict[k].update(extracted_info)

print_item_key = list(info_dict.keys())[4]

# print debug
print("Extracted Content:\n")
print(info_dict[print_item_key]['content_text'][:300]+"...\n")

print("Extracted Code Block:\n")
for code in info_dict[print_item_key]['content_codes']: 
    print(code)

Extracted Content:

前言

該系列是為了讓看過Vue官方文件或學過Vue但是卻不知道怎麼下手去重構現在有的網站而去規畫的系列文章，在這邊整理了許多我自己使用Vue重構很多網站的經驗分享給讀者們。

什麼？單元測試？當你開始接觸開發專案有一段時間後，你會開始漸漸聽到這個專業術語，就讓我來大家了解一下什麼是單元測試

影片搭配文章看學習效果會更好喔
什麼是單元測試？
簡單來說程式碼的最小單位進行測試，確保程式邏輯不會在團隊維護的過程中出錯，維護程式碼的品質。所謂的最小單位，我用個例子來舉例，假如你今天有一個主功能是由 A跟Ｂ兩個功能所組成的，而這兩個功能就是我們所說的最小單位，所以在撰寫測試的時候我們重點在針對A跟Ｂ...

Extracted Code Block:

vue add unit-jest

describe("Test to do list", () => {
  it("Test to do 1", () => {
    expect(1 + 1).toBe(2);
  });
  test("Test to do 2", () => {
   expect(2 + 1).toBe(3);
  });
});

<!-- HelloWorld.vue -->
<template>
  <h1>new message</h1>
</template>


import { shallowMount } from "@vue/test-utils";
import HelloWorld from "@/components/HelloWorld.vue";
describe("HelloWorld.vue", () => {
  it("renders msg text", () => {
    const wrapper = shallowMount(HelloWorld);
    expect(wrapper.text()).toBe("new message");
  });
});



## Preprocessing 3: Content Text
- word segmentation (use package `jieba`)
- stop word (including punctuation) removal

In [7]:
# word segmentation, stop word removal
def preprocessing_content_text(info: dict, content_text: str, stopword_set: set) -> dict: 

    # split content into line
    line_list = []
    for line in content_text.split():
        
        line = line.strip()
        if not line: 
            continue
        line_list.append(line)
    
    # process content line
    word_seg_line_list = []
    processed_line_list = []
    for line in line_list: 
        
        # word segmentation
        word_list = jieba.lcut(line, HMM=True)
        word_seg_line = " ".join(word_list)
        word_seg_line_list.append(word_seg_line)
        
        # start line processing
        processed_word_list = []
        for w in word_list: 
            
            w = w.strip().lower()
            
            # stopword removal
            if not w or w in stopword_set: 
                continue
            processed_word_list.append(w)
        processed_line = " ".join(processed_word_list)
        processed_line_list.append(processed_line)
    
    extracted_info = {
        'word_seg_content_text':           '\n'.join(word_seg_line_list),
        'word_seg_processed_content_text': '\n'.join(processed_line_list)
    }
    
    return extracted_info

In [8]:
for k in info_dict:   
    extracted_info = preprocessing_content_text(info_dict[k], info_dict[k]['content_text'], merged_stopword_set)
    info_dict[k].update(extracted_info)

# print debug
print_item_key = list(info_dict.keys())[4]

print("Word Segmentated Content:\n")
print(info_dict[print_item_key]['word_seg_content_text'][:300]+"...\n")

print("Processed Content:\n")
print(info_dict[print_item_key]['word_seg_processed_content_text'][:300]+"...\n")

Word Segmentated Content:

前言
該 系列 是 為 了 讓 看過 Vue 官方 文件 或學過 Vue 但是 卻 不 知道 怎麼 下手 去 重構 現在 有 的 網站 而 去 規畫 的 系列 文章 ， 在 這邊 整理 了 許多 我 自己 使用 Vue 重構 很多 網站 的 經驗 分享 給 讀者 們 。
什麼 ？ 單元測試 ？ 當你 開始 接觸 開發 專案 有 一段時間 後 ， 你 會 開始 漸漸 聽到 這個 專業術語 ， 就讓 我來 大家 了解 一下 什麼 是 單元測試
影片 搭配 文章 看 學習效果 會 更好 喔
什麼 是 單元測試 ？
簡單 來說 程式碼 的 最小 單位 進行 測試 ， 確保 程式 邏輯 不會 在 團隊 ...

Processed Content:

前言
系列 看過 vue 官方 文件 或學過 vue 下手 重構 網站 規畫 系列 文章 整理 vue 重構 很多 網站 經驗 分享 讀者
單元測試 當你 接觸 開發 專案 一段時間 會 漸漸 聽到 專業術語 就讓 我來 了解 單元測試
影片 搭配 文章 學習效果 會 更好 喔
單元測試
簡單 來說 程式碼 最小 單位 測試 確保 程式 邏輯 團隊 維護 過程 中 出錯 維護 程式碼 品質 最小 單位 我用 個例 子來 舉例 主 功能
a ｂ 兩個 功能 兩個 功能 所說 最小 單位 撰寫 測試 重點 針對 a ｂ 測試 主 功能 測試 中 包含
a ｂ 測試 測試 所說 單元測試
單元測試
...



## Preprocessing 4: Keyword Extraction from Processed Contents

In [9]:
def preprocessing_keyword_extraction(info: dict, kw_extract_model: Callable, word_seg_processed_content_text: str) -> dict: 
    
    kw_tup_list = kw_extract_model.extract_keywords(
        word_seg_processed_content_text, 
        keyphrase_ngram_range=(1, 1), 
        top_n=5
    )
    kw_list = [ kw_tup[0] for kw_tup in kw_tup_list ]
    
    kwp_tup_list = kw_extract_model.extract_keywords(
        word_seg_processed_content_text,
        keyphrase_ngram_range=(1, 2), 
        top_n=5
    )
    kwp_list = [ kwp_tup[0] for kwp_tup in kwp_tup_list ]
    
    print(kw_list)
    print(kwp_list)
    print()
        
    extracted_info = {
        "extracted_keywords": kw_list,
        "extracted_keywords_phrases": kwp_list
    }
    
    return extracted_info

In [10]:
# download and load pretrained model
kw_model = KeyBERT(model="paraphrase-multilingual-MiniLM-L12-v2")

# keyword extraction
for k in info_dict:
    extracted_info = preprocessing_keyword_extraction(info_dict, kw_model, info_dict[k]['word_seg_processed_content_text'])
    info_dict[k].update(extracted_info)

['2022', '參賽', '去年', '推薦', '我要']
['2022 鐵人', '我要 參賽', '2022', '參賽', '不知不覺 2022']

['stm32', '參考手冊', '手冊', '書籍', '開發板']
['stm32 書籍', 'stm32 教學', 'stm32 開發', 'stm32 開發板', '書籍 開發']

['發表', '內容', 'xd', 'x509', '負載平衡']
['xd 近期', '實際上 內容', '內容 實際上', '地盤 發表', '系列 發表']

['grid', '彈性', '空間', '定位問題', '困難']
['grid 實在話', 'grid 好像', '複習 grid', '去換 grid', '搭配 grid']

['學習效果', '課程', '單元測試', '新課程', 'testing']
['單元測試 產品開發', '撰寫 單元測試', '單元測試 影片', '文章 學習效果', '單元測試 框架']

['shadow', '陰影', '跨平台', '平台', 'css3']
['陰影 css3', 'shadow 屬性', 'box shadow', 'shadow react', '卡牌 陰影']

['swflab', 'synchronization', '合約', '鏈上', 'event']
['time conclusion', 'reference synchronization', '事件 起因', '觀察 合約', '合約 宣告']

['開發板', 'pcb', 'pcbdoc', '軟體', '電腦']
['pcb 淘寶', '淘寶 pcb', '分享 pcb', '溝通 軟體', '手機 淘寶']

['uselayouteffect', 'useeffect', '按鈕', '點擊', 'hook']
['執行 uselayouteffect', 'uselayouteffect hook', 'uselayouteffect 也許', '補充 uselayouteffect', 'useeffect uselayouteffect']

['組織', '數位', '公司', '變化趨勢', '基礎設施']
['數位 轉型', '組織 面臨'

## Format Json for Meilisearch Indexing
- Datetime String to Unix Time

- Split Tags String by `,`

- Info Dict Keys
```json
[
    'href',
    'title',
    'content_html',
    'processed_content_html',
    'content_text',
    'word_seg_content_text',
    'word_seg_processed_content_text'
    'content_codes',
    'extracted_keywords',
    'extracted_keywords_phrases',
    'raw_tags_string',
    'genre',
    'published_at',
    'author_href',
    'author_name',
    'series_href',
    'series_name',
    'series_num',
]
```

- Meilisearch Schema
    - index for document search
        ```json
        {   
            "href":                       "string",
            "title":                      "string",
            "raw_hl_content":             "string",
            "word_seg_processed_content": "string",
            "keywords":                   ["string"],
            "hashtags":                   ["string"],
            "genre":                      "string",
            "published_at":               "string",
            "published_at_unix":          "int",
            "author_href":                "string",
            "author_name":                "string",
            "series_href":                "string",
            "series_name":                "string",
            "series_num":                 "int"
            "reading_time":               "int"
        }
        ```
    - index for auto fill
        ```json
        {
            "phrases": "string"
        }
        ```

In [11]:
def date_str_to_unix_time(time_str:str, format_str: str) -> int: 
    return int(time.mktime(datetime.datetime.strptime(time_str, format_str).timetuple()))

def count_chinese_tech_article_reading_time(content: str, code_list: list) -> int: 
    return max(len(content)//300, 1) + len(code_list)

In [12]:
doc_search_res_dict_list = []
keyword_search_res_dict_list = []

for k in info_dict:
    
    doc_search_res_dict = {
        'href':                       info_dict[k]['href'],
        'title':                      info_dict[k]['title'],
        'raw_hl_content':             info_dict[k]['content_text'],
        'word_seg_processed_content': info_dict[k]['word_seg_processed_content_text'],
        'keywords':                   info_dict[k]['extracted_keywords'],
        'hashtags':                   info_dict[k]['raw_tags_string'].split(','),
        'genre':                      info_dict[k]['genre'],
        'published_at':               info_dict[k]['published_at'],
        'published_at_unix':          date_str_to_unix_time(info_dict[k]['published_at'], "%Y-%m-%d %H:%M:%S"),
        'author_href':                info_dict[k]['author_href'],
        'author_name':                info_dict[k]['author_name'],
        'series_href':                info_dict[k]['series_href'],
        'series_name':                info_dict[k]['series_name'],
        'series_num':                 info_dict[k]['series_num'],
        'reading_time':               count_chinese_tech_article_reading_time(
                                          info_dict[k]['content_text'],
                                          info_dict[k]['content_codes']
                                      )
    }
    doc_search_res_dict_list.append(doc_search_res_dict)
    
    keyword_search_res_dict = {
        'phrases':                    info_dict[k]['extracted_keywords_phrases'],
    }
    keyword_search_res_dict_list.append(keyword_search_res_dict)


# print debug
print(json.dumps(doc_search_res_dict_list[4], indent=4, ensure_ascii=False))
print(json.dumps(keyword_search_res_dict_list[4], indent=4, ensure_ascii=False))

{
    "href": "https://ithelp.ithome.com.tw/articles/10282236",
    "title": "[專案上線第01天] -  新來的主管說要寫 Vue Test Utils 單元測試",
    "raw_hl_content": "前言\n\n該系列是為了讓看過Vue官方文件或學過Vue但是卻不知道怎麼下手去重構現在有的網站而去規畫的系列文章，在這邊整理了許多我自己使用Vue重構很多網站的經驗分享給讀者們。\n\n什麼？單元測試？當你開始接觸開發專案有一段時間後，你會開始漸漸聽到這個專業術語，就讓我來大家了解一下什麼是單元測試\n\n影片搭配文章看學習效果會更好喔\n什麼是單元測試？\n簡單來說程式碼的最小單位進行測試，確保程式邏輯不會在團隊維護的過程中出錯，維護程式碼的品質。所謂的最小單位，我用個例子來舉例，假如你今天有一個主功能是由 A跟Ｂ兩個功能所組成的，而這兩個功能就是我們所說的最小單位，所以在撰寫測試的時候我們重點在針對A跟Ｂ來進行測試，主功能的測試中不會包含 A跟Ｂ的測試，這樣的測試就是我們所說的單元測試。\n為什麼需要單元測試？\n我先列出幾個優缺點，我們來比較一下\n優點：\n\n確保團隊跌代的時候不會影響原本的功能\n確保品質，準確對程式碼切割最小單位，降低耦合度\n程式的 return 變成可預期\n重構程式碼可以按照測試的規格走\n\n缺點 :\n\n初期可能撰寫單元測試所消耗的時間可能會大於實際開發時間\n如果迭代性高，很常寫好的測試要重寫，久了會浪費很多時間\n測試相關的配置繁瑣，為了配合許多開發上的細節要處理的設定很多\n\n從優缺點可以知道說，撰寫單元測試的初期對於開發的效益並不高，先撇開不熟悉測試來說，光就在公司常常會因為需求改變就要來來回回改寫程式，我就要再花許多時間來重寫測試，怎麼想都對於有時辰壓力的專案來說不是那麼划算，所以往往會在這個時候放棄寫測試，就像我如果在初期會遇到不斷修改的需求的時候，我也不會先寫測試（笑\n那什麼情況下該寫單元測試？其實產品開發的中期的時候，基本上中期的時候大多數的平台規格都確定了差不多，就可以考慮開始補單元測試，因為會開始遇到前面做好的功能因為新的功能造成預期外的錯誤，以及專案由其他同事接手的時候改壞某個功能但是同事不知道，這些我們都可以透

## Dump Json for Meilisearch Indexing Testing

In [13]:
with open(os.path.join(nlp_root, "testing/testing-meilisearch_docs_indexing.json"), 'w') as ofile: 
    ofile.write(json.dumps(doc_search_res_dict_list, indent=4, ensure_ascii=False))
    
with open(os.path.join(nlp_root, "testing/testing-meilisearch_keywords_indexing.json"), 'w') as ofile: 
    ofile.write(json.dumps(keyword_search_res_dict_list, indent=4, ensure_ascii=False))